In [143]:
from satelite_data_preporation import collection
import ee 
import geemap
ee.Initialize()
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
%autoreload 2

In [145]:
begin = '2019-05-15'
end =  '2019-08-15'

In [146]:
two_fields = 'vector_data/archangelskoe_TWO_FIELDS_WGS84.geojson'

In [187]:
region_boundary = 'vector_data/budenovsk_district_boundaries.geojson'
region_of_interest = 'vector_data/our_fields_for_animation.geojson'

sentinel2021_collection = collection('sentinel2', begin, end, region_boundary, region_of_interest)

In [188]:
sentinel2021_collection.DownloadImages()
sentinel2021_collection.get_raster_plots()

Generating URL ...
Please wait ...
Data downloaded to c:\Users\ermolaevNR\Google Диск\python_notebooks\Archangelskiy_project\field_scale\raster_data\Field_scale_sentinel2_2019-05-15_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ermolaevNR\Google Диск\python_notebooks\Archangelskiy_project\field_scale\raster_data\Field_scale_sentinel2_2019-05-30_scene.tif
file Field_scale_sentinel2_2019-06-09_scene.tif alredy exists
file Field_scale_sentinel2_2019-06-14_scene.tif alredy exists
file Field_scale_sentinel2_2019-07-04_scene.tif alredy exists
file Field_scale_sentinel2_2019-07-19_scene.tif alredy exists
file Field_scale_sentinel2_2019-07-29_scene.tif alredy exists
file Field_scale_sentinel2_2019-08-03_scene.tif alredy exists
file Field_scale_sentinel2_2019-08-08_scene.tif alredy exists
file Field_scale_sentinel2_2019-08-13_scene.tif alredy exists


sentinel2021_collection.anual_ndvi()

In [151]:
NDVI_anual = sentinel2021_collection.anual_ndvi(two_fields)

file anual_data/NDVI/NDVI_modis_2019-05-15_2019-08-15.csv alredy exists


In [163]:
url = "https://rp5.ru/%D0%90%D1%80%D1%85%D0%B8%D0%B2_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D1%8B_%D0%B2_%D0%91%D1%83%D0%B4%D0%B5%D0%BD%D0%BD%D0%BE%D0%B2%D1%81%D0%BA%D0%B5"
climat = sentinel2021_collection.anual_weather(url)

file anual_data/Weather/weather_2019-05-15_2019-08-15.csv alredy exists


In [184]:
sentinel2021_collection.get_raster_plots()

In [32]:

weather_data = pd.read_csv('anual_data/weather.csv.gz', compression= 'gzip',comment='#', sep = ';')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte

In [141]:
from parsing_climat import get_weather, agregate_temp_and_precipitation

In [27]:
url = "https://rp5.ru/%D0%90%D1%80%D1%85%D0%B8%D0%B2_%D0%BF%D0%BE%D0%B3%D0%BE%D0%B4%D1%8B_%D0%B2_%D0%91%D1%83%D0%B4%D0%B5%D0%BD%D0%BD%D0%BE%D0%B2%D1%81%D0%BA%D0%B5"
link = get_weather("03.10.2021", "04.10.2021", url)
print(link)

http://93.90.217.251/download/files.synop/37/37061.03.10.2021.04.10.2021.1.0.0.ru.ansi.00000000.csv.gz


In [115]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')


In [142]:
agregate_temp_and_precipitation(link)

,date,RRR
0,03.10.2021,0.0
1,04.10.2021,14.0
